In [38]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings("ignore")

## Dataset Pre-processing

In [73]:
TASKS = ["climbing","jumping","running"]
LABELS = {
    "climbing" : 1,
    "jumping" : 2,
    "running" : 3,
}
SENSORS = ["Accelerometer","Gyroscope"]
dataset_path = "dataset/train/"

In [83]:
data = {}
for sensors in SENSORS:
    data[sensors] = []
for vals in TASKS:
    sub_dirs = [x for x in os.walk(dataset_path + vals)]
    datapoints = sub_dirs[0][1]
    for pts in datapoints:
        for sensors in SENSORS:
            file_path = dataset_path + vals + "/" + pts + "/" +sensors + ".csv"
            df = pd.read_csv(file_path)
            df["Label"] = LABELS[vals]
            data[sensors].append(df)

In [84]:
acc = data["Accelerometer"]
gyro = data["Gyroscope"]

In [86]:
acc

[                Timestamp  Milliseconds         X         Y          Z  Label
 0     2019-10-30 21:07:32             1 -0.860009  2.308558  13.183083      1
 1     2019-10-30 21:07:32            11 -0.701998  1.705245  11.902240      1
 2     2019-10-30 21:07:32            20 -1.374740  1.571175  10.786590      1
 3     2019-10-30 21:07:32            30 -1.417834  2.208005  11.193587      1
 4     2019-10-30 21:07:32            41 -0.826491  1.446682  10.528028      1
 ...                   ...           ...       ...       ...        ...    ...
 3020  2019-10-30 21:08:02         30200  5.307190  5.564532   4.971324      1
 3021  2019-10-30 21:08:02         30210  5.577723  5.784789   5.423809      1
 3022  2019-10-30 21:08:02         30221  5.592088  5.796760   5.936146      1
 3023  2019-10-30 21:08:02         30230  5.477171  5.801547   6.079792      1
 3024  2019-10-30 21:08:02         30240  5.383801  5.811124   6.352719      1
 
 [3025 rows x 6 columns],
                 Timesta